In [1]:
# !pip install scikit-surprise

In [2]:
import surprise

In [3]:
print(surprise.__version__)

1.1.1


In [4]:
from surprise import SVD # 고유값분해(행렬을 더 낮은 차원으로 분해 but 고유한 성질은 그대로)해서 다시 원본으로 살리면서 잠재적인 값을 추론하는 방법
from surprise import Dataset, Reader # SVD를 사용하기 위한 데이터셋 만들어주는 클래스
                             # 사용자, 아이템, 평점
from surprise import accuracy # RMSE, MSE, CrossValidation(k-fold)
from surprise.model_selection import train_test_split # 훈련/검증 데이터 분류

In [5]:
# 1. 데이터셋을 만들어주자.(사용자, 아이템, 평점), 훈련/검증 데이터 분류
data = Dataset.load_builtin('ml-100k')
data # 객체로 다운받음

In [6]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

In [7]:
trainset # object

In [8]:
testset[:5] # tuple list

[('120', '282', 4.0),
 ('882', '291', 4.0),
 ('535', '507', 5.0),
 ('697', '244', 5.0),
 ('751', '385', 4.0)]

In [9]:
# 2. SVD() 모델 선정
algo = SVD()

In [10]:
# 3. 훈련용 데이터로 fit()
algo.fit(trainset)

In [11]:
# 4. 검증용 데이터로 예측
predictions = algo.test(testset)[:5]
predictions

[Prediction(uid='120', iid='282', r_ui=4.0, est=3.6388811938195973, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.676526366304428, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.067073766873423, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.485580397309596, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.57002262216569, details={'was_impossible': False})]

In [12]:
[(pred.uid, pred.iid, pred.est) for pred in predictions]

[('120', '282', 3.6388811938195973),
 ('882', '291', 3.676526366304428),
 ('535', '507', 4.067073766873423),
 ('697', '244', 3.485580397309596),
 ('751', '385', 3.57002262216569)]

In [13]:
# 5. 정확도 계산


# CSV files

In [14]:
import pandas as pd

In [15]:
# csv를 SVD하기 위한 Dataset으로 변경
# 1) csv => dataframe
# 2) dataframe => dataset

In [16]:
# csv 읽어오기 + dataframe으로
ratings = pd.read_csv('./csv_data_files/ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [17]:
ratings.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [18]:
reader = Reader(rating_scale=(0.5, 5.0))

In [19]:
# SVD에서 사용할 수 있는 데이터셋으로 만들어주자.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data

In [20]:
# 훈련/검증 데이터 분류
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

In [21]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='./csv_data_files/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [22]:
# 객체 생성
svd = SVD(n_factors=50, n_epochs= 40,  random_state=0)

# 훈련
svd.fit(trainset) 

# 검증
predictions = svd.test(testset)

# 결과분석
accuracy.rmse(predictions)

RMSE: 1.0388


1.0387802469854106

In [23]:
from surprise.model_selection import cross_validate

In [24]:
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8745  0.8880  0.8726  0.8759  0.8755  0.8773  0.0055  
MAE (testset)     0.6701  0.6811  0.6671  0.6723  0.6696  0.6720  0.0048  
Fit time          8.99    8.10    8.62    8.97    8.65    8.67    0.32    
Test time         0.20    0.30    0.27    0.21    0.31    0.26    0.05    


{'test_rmse': array([0.87446067, 0.88801124, 0.87259687, 0.87590544, 0.87553769]),
 'test_mae': array([0.67006376, 0.68114163, 0.66709887, 0.67226511, 0.66956777]),
 'fit_time': (8.992751121520996,
  8.103240251541138,
  8.617480039596558,
  8.967645168304443,
  8.645222902297974),
 'test_time': (0.20069599151611328,
  0.3045201301574707,
  0.27380919456481934,
  0.20774388313293457,
  0.3095989227294922)}

In [25]:
import time

In [26]:
start = time.time()
start

1640244518.293364

In [27]:
end = time.time()
end

1640244518.330501

In [28]:
end-start

0.03713703155517578

In [29]:
end2 = time.time()
end2

1640244518.573578

In [30]:
end2-start

0.2802138328552246

# 모든 영화 csv 데이터로 예측

In [31]:
# 영화에 대한 상세 속성 정보 dataframe 로딩
movies = pd.read_csv('./csv_data_files/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [32]:
## 내가 안본 영화 리스트 구해서, 이중에서 추천하려고 함.
def get_unseen_surprise(movies, ratings, userId):
    
    # 전체 영화id 리스트
    total_movies = movies['movieId'].tolist()
    
    # 내가 본 영화id 리스트
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
    
    # 추천 대상이 되는 영화 리스트: 전체 영화 리스트 - 내가 본 영화 리스트
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    
    print("전체 영화 리스트 개수 >> ", len(total_movies))
    print("내가 본 영화 리스트 개수 >> ", len(seen_movies))
    print("내가 안 본 영화 리스트 개수 >> ", len(unseen_movies))

    return unseen_movies

In [33]:
unseen_movies = get_unseen_surprise(movies, ratings, 9)

전체 영화 리스트 개수 >>  9742
내가 본 영화 리스트 개수 >>  46
내가 안 본 영화 리스트 개수 >>  9696


In [34]:
## 안본 영화중에서 평점 예측이 높게 나온 3개를 리스트업하는 함수
def recomm_movie_by_surprise(svd, userId, unseen_movies, top_n=5):
    
    # 안 본 영화리스트를 하나씩 꺼낸 다음 평점을 예측하세요
    predictions = [svd.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # 평점이 높은 순으로 정렬하는 기준을 함수화
    def sortkey_est(one):
        return one.est
    
    # 평점이 높은 순으로 정렬해서 top10 추천
    predictions.sort(key=sortkey_est,reverse=True)
    top_predictions = predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_rating = [pred.est for pred in top_predictions]
    print(movies)
    top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

In [35]:
recomm_movie_by_surprise(svd, 9, unseen_movies, top_n=10)

      movieId                                      title  \
0           1                           Toy Story (1995)   
1           2                             Jumanji (1995)   
2           3                    Grumpier Old Men (1995)   
3           4                   Waiting to Exhale (1995)   
4           5         Father of the Bride Part II (1995)   
...       ...                                        ...   
9737   193581  Black Butler: Book of the Atlantic (2017)   
9738   193583               No Game No Life: Zero (2017)   
9739   193585                               Flint (2017)   
9740   193587        Bungo Stray Dogs: Dead Apple (2018)   
9741   193609        Andrew Dice Clay: Dice Rules (1991)   

                                           genres  
0     Adventure|Animation|Children|Comedy|Fantasy  
1                      Adventure|Children|Fantasy  
2                                  Comedy|Romance  
3                            Comedy|Drama|Romance  
4                  

[(1, 'Toy Story (1995)', 3.501685901647473),
 (2, 'Jumanji (1995)', 3.501685901647473),
 (3, 'Grumpier Old Men (1995)', 3.501685901647473),
 (4, 'Waiting to Exhale (1995)', 3.501685901647473),
 (5, 'Father of the Bride Part II (1995)', 3.501685901647473),
 (6, 'Heat (1995)', 3.501685901647473),
 (7, 'Sabrina (1995)', 3.501685901647473),
 (8, 'Tom and Huck (1995)', 3.501685901647473),
 (9, 'Sudden Death (1995)', 3.501685901647473),
 (10, 'GoldenEye (1995)', 3.501685901647473)]